# Toy Radix Lagrangean decomposition

In [1]:
SCALE_CROWDING=False
PREVENT_ZERO = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f38ddeff6c8>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
mdl_ref = ijomc
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound0 = 0.001
crowding_bound = crowding_bound0

not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

## Need to allow higher growth for acetate with higher uptake rate

In [7]:
df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

In [8]:
from dynamicme.decomposition import LagrangeSubmodel, LagrangeMaster
from dynamicme.generate import copy_model

In [9]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = 1.
    sn.objective_coefficient = 1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------
    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:    
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    sub = LagrangeSubmodel(gap, cond)
    sub_dict[cond] = sub
        
master = LagrangeMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [10]:
master.add_submodels(sub_dict)

In [ ]:
#sub._ys[0]
#y.Start
#x.Start

In [ ]:
#%%time

max_iter = 1000
gaptol = 1e-3

master.model.Params.OutputFlag =0
#master.model.Params.Presolve = 0
#master.model.Params.ScaleFlag = 0

print_int = 10

UB = 1e15
LB = -1e15
delta = 1.

_iter = 0.
while _iter < max_iter:
    # Solve master
    master.model.update()
    master.model.optimize()
#     print("Solved master with status: %s. Optimal: %s" % (master.model.Status, master.model.Status==GRB.OPTIMAL))
    uk = np.array([u.X for u in master._us])
    sub_objs = []
    for sub_ind,sub in iteritems(master.sub_dict):
        sub.update_obj(uk)
        #sub.model.Params.OutputFlag = 1
        if sub.model.Status==GRB.OPTIMAL:
            xk = [x.X for x in sub._xs]
            yk = [y.X for y in sub._ys]
            sub.optimize(xk,yk)
        else:
            sub.model.optimize()
        yk = [y.X for y in sub._ys]
        xk = [x.X for x in sub._xs]
        cut = master.make_optcut(sub,xk,yk)
        master.model.addConstr(cut)        
        
        sub_obj = sub.model.ObjVal
        sub_objs.append(sub_obj)
#         print("Solved sub %s with status: %s. Optimal: %s" % (sub_ind, sub.model.Status, sub.model.Status==GRB.OPTIMAL))    
    LB = sum(sub_objs)
    UB = master._z.X
    gap = UB-LB
    relgap = gap/UB
    if np.mod(_iter, print_int)==0:
        print("Iter: %d. Gap: %g = %g - %g (relgap=%.3g%%)" % (_iter,gap,UB,LB, relgap*100))
    
    if relgap < gaptol:
        print("Gap < Gaptol at iter %d"%_iter)
        break
    else:
        delta = max(delta/2., 1e-10)
        master.update_obj(delta)
    _iter+=1

Iter: 0. Gap: 999.828 = 1000 - 0.172158 (relgap=100%)
Iter: 10. Gap: 43.8955 = 0.54366 - -43.3518 (relgap=8.07e+03%)
Iter: 20. Gap: 26.3212 = 0.469625 - -25.8516 (relgap=5.6e+03%)
Iter: 30. Gap: 25.5961 = 0.46961 - -25.1265 (relgap=5.45e+03%)
Iter: 40. Gap: 36.054 = 0.46961 - -35.5844 (relgap=7.68e+03%)
Iter: 50. Gap: 4.73746 = 0.469609 - -4.26785 (relgap=1.01e+03%)
Iter: 60. Gap: 50.0074 = 0.46953 - -49.5378 (relgap=1.07e+04%)
Iter: 70. Gap: 48.9497 = 0.469429 - -48.4803 (relgap=1.04e+04%)
Iter: 80. Gap: 95.4232 = 0.466514 - -94.9567 (relgap=2.05e+04%)
Iter: 90. Gap: 9.06701 = 0.460983 - -8.60603 (relgap=1.97e+03%)
Iter: 100. Gap: 5.26683 = 0.451384 - -4.81544 (relgap=1.17e+03%)
Iter: 110. Gap: 3.39903 = 0.438276 - -2.96075 (relgap=776%)
Iter: 120. Gap: 4.97972 = 0.428685 - -4.55103 (relgap=1.16e+03%)
Iter: 130. Gap: 2.31021 = 0.402362 - -1.90785 (relgap=574%)
Iter: 140. Gap: 1.44164 = 0.389183 - -1.05246 (relgap=370%)


## Are all yks the same?

In [ ]:
yids = [y.VarName for y in sub._ys]
key0 = master.sub_dict.keys()[0]
sub0 = master.sub_dict[key0]
yk0  = np.array([sub0.model.getVarByName(yid).X for yid in yids])

err_dict = {}
for sub_ind,sub in iteritems(master.sub_dict):
    yk = np.array([sub.model.getVarByName(yid).X for yid in yids])    
    err_dict[sub_ind] = sum(yk != yk0)

In [ ]:
err_dict

yopt = [sol_master[y.VarName] for y in master._ys]
print('Number of non-zero binaries: %g' % sum(yopt))
# Fitted parameters

kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
#             if abs(y)>1e-10:
#                 print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
kfit_changed

#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:
    ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')    
    crowding = Constraint('crowding')
    crowding._bound = crowding_bound0
    crowding._constraint_sense = 'L'
    
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit})
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))